In [103]:
# constants
DATA_PATH='../data/raw/survey_results_public.csv'
EXPORT_PATH = "../data/processed/1_preprocessed_data.pkl"

REPLACE_DICT={
    'YearsCode':{'Less than 1 year':0,'More than 50 years':51 },
    'YearsCodePro':{'Less than 1 year':0,'More than 50 years':51 },
}

In [104]:
# packages
import pandas as pd 
import numpy as np
import logging
import pickle

# Functions

In [105]:
def Replace_nulls(pd_series,replacement=[]):
    '''
        Aim:
        Replace null values with a specific replacement
        
        Parmeters:
        * pd_series (pd.Series): String series with answers
        * replacement (list) : list to replace null value
        
        Return: (pd.Series): series after replacing 
    '''
    nulls=pd_series.isnull()
    pd_series.loc[nulls]=pd_series.loc[nulls].apply(lambda x: replacement)
    return pd_series

In [106]:
def split_multianswers(answers,delimit=';'):
    '''
        Aim:
        Split multiple answers in a single string 
        to a list of strings each represnting a single answer
        
        Parameters:
        * answers (pd.Series): String series with answers 
        * delimit (string): endpoint for each single answer Defaults to ";"
        
        Return: (pd.Series): If column contains multiple answers
    
    '''
     # Sub functions 
    def is_splittable(answer,delimit):
        """ Check if answer should be splitted - Returns boolean """    
        return answer.str.contains(delimit)
    
    def split_answer(answer,delimit):
        """Function to split single answer - Returns list of strings"""
        return answer.str.split(delimit)
    
    #__________________________________________________________#
    
    # Check if multiple answers exist - if none: return original 
    splittable_answers=is_splittable(answers,delimit)
    if not splittable_answers.any():
        return answers
    
    # Split each multiple answer to list of single answers
    splitted_answers=split_answer(answers,delimit)
    
    # Replace NAs with empty lists 
    modified_values=Replace_nulls(splitted_answers)
    
    return modified_values
    

##### ________________________________________________________________________________________________________________________

# Preprocessing

In [107]:
# Read data
raw_data=pd.read_csv(DATA_PATH)
data=raw_data.copy()

## Replace values

In [108]:
for col,vals in REPLACE_DICT.items():
    data[col]=data[col].replace(vals).astype(np.float32)
    
# we applied parsing by changing type to float32

## Split multiple values

In [109]:
object_cols = data.select_dtypes(include='object').columns.tolist()
for col in object_cols:
    
    data[col]=split_multianswers(data[col])

##### ________________________________________________________________________________________________________________________

# Visually verify results

In [110]:
idx=data.sample(1).index[0]

print(raw_data['LanguageWantToWorkWith'].iloc[idx])
print(data['LanguageWantToWorkWith'].iloc[idx])

Go;Java;Kotlin;SQL
['Go', 'Java', 'Kotlin', 'SQL']


In [111]:
idx=data.sample(1).index[0]

print(raw_data['DevType'].iloc[idx])
print(data['DevType'].iloc[idx])

Developer, desktop or enterprise applications
['Developer, desktop or enterprise applications']


In [112]:
idx=data.sample(1).index[0]

print(raw_data['YearsCode'].iloc[idx])
print(data['YearsCode'].iloc[idx])

2
2.0


##### ________________________________________________________________________________________________________________________

# Export data

In [101]:
data.to_pickle(EXPORT_PATH)

In [102]:
pk=pd.read_pickle(EXPORT_PATH)
pk

,ResponseId,MainBranch,Employment,Country,US_State,UK_Country,EdLevel,Age1stCode,LearnCode,YearsCode,...,Age,Gender,Trans,Sexuality,Ethnicity,Accessibility,MentalHealth,SurveyLength,SurveyEase,ConvertedCompYearly
0,1,I am a developer by profession,"Independent contractor, freelancer, or self-em...",Slovakia,NaN,NaN,"Secondary school (e.g. American high school, G...",18 - 24 years,"[Coding Bootcamp, Other online resources (ex: ...",NaN,...,25-34 years old,[Man],No,[Straight / Heterosexual],[White or of European descent],[None of the above],[None of the above],Appropriate in length,Easy,62268.0
1,2,I am a student who is learning to code,"Student, full-time",Netherlands,NaN,NaN,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",11 - 17 years,"[Other online resources (ex: videos, blogs, et...",7.0,...,18-24 years old,[Man],No,[Straight / Heterosexual],[White or of European descent],[None of the above],[None of the above],Appropriate in length,Easy,NaN
2,3,"I am not primarily a developer, but I write co...","Student, full-time",Russian Federation,NaN,NaN,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",11 - 17 years,"[Other online resources (ex: videos, blogs, et...",NaN,...,18-24 years old,[Man],No,[Prefer not to say],[Prefer not to say],[None of the above],[None of the above],Appropriate in length,Easy,NaN
3,4,I am a developer by profession,Employed full-time,Austria,NaN,NaN,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",11 - 17 years,[],NaN,...,35-44 years old,[Man],No,[Straight / Heterosexual],[White or of European descent],[I am deaf / hard of hearing],[],Appropriate in length,Neither easy nor difficult,NaN
4,5,I am a developer by profession,"Independent contractor, freelancer, or self-em...",United Kingdom of Great Britain and Northern I...,NaN,England,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",5 - 10 years,[Friend or family member],17.0,...,25-34 years old,[Man],No,[],[White or of European descent],[None of the above],[],Appropriate in length,Easy,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83434,83435,I am a developer by profession,Employed full-time,United States of America,Texas,NaN,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",11 - 17 years,"[Other online resources (ex: videos, blogs, et...",6.0,...,25-34 years old,[Man],No,[Straight / Heterosexual],[White or of European descent],[None of the above],[I have a concentration and/or memory disorder...,Appropriate in length,Easy,160500.0
83435,83436,I am a developer by profession,"Independent contractor, freelancer, or self-em...",Benin,NaN,NaN,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",11 - 17 years,"[Other online resources (ex: videos, blogs, et...",4.0,...,18-24 years old,[Man],No,[Straight / Heterosexual],[Black or of African descent],[None of the above],[None of the above],Appropriate in length,Easy,3960.0
83436,83437,I am a developer by profession,Employed full-time,United States of America,New Jersey,NaN,"Secondary school (e.g. American high school, G...",11 - 17 years,[School],10.0,...,25-34 years old,[Man],No,[],[White or of European descent],[None of the above],[None of the above],Appropriate in length,Neither easy nor difficult,90000.0
83437,83438,I am a developer by profession,Employed full-time,Canada,NaN,NaN,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",11 - 17 years,"[Online Courses or Certification, Books / Phys...",5.0,...,25-34 years old,[Man],No,[Straight / Heterosexual],[White or of European descent],[None of the above],[I have a mood or emotional disorder (e.g. dep...,Appropriate in length,Neither easy nor difficult,816816.0
